In [1]:
import sys

#import Kalpana functions from github repository
# change these to your own paths!
sys.path.append('/Users/vivianhuang/Documents/GitHub/Kalpana')
from kalpana.export import *
from kalpana.visualizations import *
import contextily as cx
import os


In [2]:
#plot the maximum water levels from a given maxele.63.nc file

## path to netcdf file
### netcdf is just a way to store datasets
nc_path_2 = '/Users/vivianhuang/Documents/GitHub/Kalpana/examples/test/fort.63.nc' 
nc2 = netcdf.Dataset(nc_path_2, 'r')
#check if this netcdf file looks right
print(nc2)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    _FillValue: -99999.0
    model: ADCIRC
    version: v55.00-49-ga2c71f6
    grid_type: Triangular
    description: CTXCS_TP_0012_HIS                         ! 32 CHARACTER ALPHANUMERIC RUN DESCRI
    agrid: Modified fort.14
    rundes: CTXCS_TP_0012_HIS                         ! 32 CHARACTER ALPHANUMERIC RUN DESCRI
    runid: Tides_1_SLC_0_RFC_0_WAV_1_GCP_CTX61LE02                          ! 24 CHARACTER
    title: namo
    institution: namo
    source: namo
    history: namo
    references: namo
    comments: namo
    host: namo
    convention: namo
    Conventions: UGRID-0.9.0
    contact: namo
    creation_date: 2023-03-23 14:52:27 -05:00
    modification_date: 2023-03-23 14:52:27 -05:00
    fort.15: ==== Input File Parameters (below) ====
    dt: 0.125
    ihot: 0
    ics: 2
    nolibf: 1
    nolifa: 2
    nolica: 1
    nolicat: 1
    nwp: 7
    ncor: 1
    ntip: 0
    nws: 0
    nra

In [3]:
def runExtractContours63(ncObj, var, levels, conType, epsg, stepLevel, orgMaxLevel, dzFile=None, zeroDif=-20,time_index=0):
    ''' Run "contours2gpd" or "filledContours2gpd" if npro = 1 or "contours2gpd_mp" or "filledContours2gpd_mp" if npro > 1.
        Parameters
            ncObj: netCDF4._netCDF4.Dataset
                Adcirc input file
            var: string
                Name of the variable to export
            levels: np.array
                Contour levels. The max value in the entire doman and over all timesteps is added to the requested levels.
            conType: string
                'polyline' or 'polygon'
            epsg: int
                coordinate system
            stepLevel: int or float
                step size of the levels requested
            orgMaxLevel: int or float
                max level requested
            dzFile: str
                full path of the pickle file with the vertical difference between datums
                for each mesh node
            zeroDif: int
                threshold for using nearest neighbor interpolation to change datum. Points below
                this value won't be changed.
        Returns
            gdf: GeoDataFrame
                Polygons or polylines as geometry columns. If the requested file is time-varying the GeoDataFrame will include all timesteps.
            
    '''
    ## get triangles and nodes coordinates
    nv = ncObj['element'][:,:] - 1 ## triangles starts from 1
    x = ncObj['x'][:].data
    y = ncObj['y'][:].data
    z = ncObj['depth'][:].data
    
    ## get extra info: variable name, variable long-name and unit name
    vname = ncObj[var].name
    lname = ncObj[var].long_name
    #u = ncObj[var].units

    ## matplotlib triangulation
    tri = mpl.tri.Triangulation(x, y, nv)
    
    ## if the variable requested is the bathymetry, values are inverted (times -1) for plotting
    if var == 'depth':
        timeVar = 0
        auxMult = -1
    else:
        auxMult = 1
    
    ## time constant

    aux = ncObj[var][time_index][:].data
    if dzFile != None: ## change datum
        dfNewDatum = changeDatum(x, y, z, aux, dzFile, zeroDif)
        ## change nan to -99999 and transform it to a 1D vector
        aux = np.nan_to_num(dfNewDatum['newVar'].values, nan = -99999.0).reshape(-1)*auxMult
    else: ## original datum remains constant
        ## change nan to -99999 and transform it to a 1D vector
        aux = np.nan_to_num(aux, nan = -99999.0).reshape(-1)*auxMult
    ## non-filled contours
    if conType == 'polyline':
        labelCol = 'z'
        gdf = contours2gpd(tri, aux, levels, epsg, True)
    ## filled contours
    elif conType == 'polygon':
        labelCol = 'zMean'
        gdf = filledContours2gpd(tri, aux, levels, epsg, stepLevel, orgMaxLevel, True)
    ## error message
    else:
        print('only "polyline" and "polygon" types are supported!')
        sys.exit(-1)
    ## add more info to the geodataframe
    gdf['variable'] = [vname]*len(gdf)
    gdf['name'] = [lname]*len(gdf)
        #gdf['zLabelCol'] = [f'{x:0.2f} {unit}' for x in gdf[labelCol]]        
    return gdf

In [4]:
def nc2shp63(ncFile, var, levels, conType, pathOut, epsgOut, vUnitOut='ft', vUnitIn='m', epsgIn=4326,
           subDomain=None, epsgSubDom=None, exportMesh=False, meshName=None, dzFile=None, zeroDif=-20,time_index=0):
    ''' Run all necesary functions to export adcirc outputs as shapefiles.
        Parameters
            ncFile: string
                path of the adcirc output, must be a netcdf file
            var: string
                Name of the variable to export
            levels:list
                Contour levels. Min, Max and Step. Max IS included as in np.arange method.
                Values must be in vUnitOut vertical unit.
            conType: string
                'polyline' or 'polygon'
            pathout: string
                complete path of the output file (*.shp or *.gpkg)
            epsgOut: int
                coordinate system of the output shapefile
            vUnitIn, vUnitOut: string. Default for vUnitIn is 'm' and 'ft' for vUnitOut
                input and output vertical units. For the momment only supported 'm' and 'ft'
            epsgIn: int. Default 4326.
                coordinate system of the adcirc input
            subDomain: str or list. Default None
                complete path of the subdomain polygon kml or shapelfile, or list with the
                uper-left x, upper-left y, lower-right x and lower-right y coordinates. The crs must be the same of the
                adcirc input file.
            exportMesh: boolean. Default False
                True for export the mesh geodataframe and also save it as a shapefile
            meshName: str
                file name of the output mesh shapefile
            dzFile: str
                full path of the pickle file with the vertical difference between datums
                for each mesh node
            zeroDif: int
                threshold for using nearest neighbor interpolation to change datum. Points below
                this value won't be changed.
        Returns
            gdf: GeoDataFrame
                gdf with contours
            mesh: GeoDataFrame, only if exportMesh is True
                gdf with mesh elements, representative length and area of each triangle
    '''
    
    print('Start exporting adcirc to shape')
    ## read adcirc file
    nc = netcdf.Dataset(ncFile, 'r')
    ## change units of the requested levels
    if vUnitIn == 'm' and vUnitOut == 'ft':
        levels = [l / 3.2808399 for l in levels]
    elif vUnitIn == 'ft' and vUnitOut == 'm':
        levels = [l * 3.2808399 for l in levels]
    if conType == 'polygon':   
        maxmax = np.max(nc[var][:].data)
        orgMaxLevel = levels[1]
        stepLevel = levels[2]
        ## list of levels to array
        levels_aux = np.arange(levels[0], np.ceil(maxmax), stepLevel)
        ## given levels will now match the avarege value of each interval    
        levels_aux = levels_aux - stepLevel/2
        levels = levels_aux.copy()
    else:
        orgMaxLevel = levels[1]
        stepLevel = levels[2]
        levels = np.arange(levels[0], orgMaxLevel + stepLevel, stepLevel)
    
    t00 = time.time()
    gdf = runExtractContours63(nc, var, levels, conType, epsgIn, stepLevel, orgMaxLevel, 
                            dzFile, zeroDif,time_index)
    print(f'    Ready with the contours extraction: {(time.time() - t00)/60:0.3f} min')
    
    ## clip contours if requested
    if subDomain is not None:
        t0 = time.time()
        subDom = readSubDomain(subDomain, epsgSubDom)
        gdf = gpd.clip(gdf, subDom.to_crs(epsgIn))
        print(f'    Cliping contours based on mask: {(time.time() - t0)/60:0.3f} min')
    
    ## change vertical units if requested
    if vUnitIn == vUnitOut:
        pass
    else:
        t0 = time.time()
        gdf = gdfChangeVerUnit(gdf, vUnitIn, vUnitOut)
        print(f'    Vertical units changed: {(time.time() - t0)/60:0.3f} min')
    
    ## change CRS if requested
    if epsgIn == epsgOut:
        pass
    else:
        t0 = time.time()
        gdf = gdf.to_crs(epsgOut)
        print(f'    Changing CRS: {(time.time() - t0)/60:0.3f} min')
    
    ## save output shape file
    t0 = time.time()
    if pathOut.endswith('.shp'):
        gdf.to_file(pathOut)
    elif pathOut.endswith('.gpkg'):
        gdf.to_file(pathOut, driver = 'GPKG')
    elif pathOut.endswith('.wkt'):
        gdf.to_csv(pathOut)
    print(f'    Saving file: {(time.time() - t0)/60:0.3f} min')
    
    ## export mesh if requested
    if exportMesh == True:
        print('    Exporting mesh')
        t0 = time.time()
        mesh = mesh2gdf(nc, epsgIn, epsgOut)
        
        if subDomain is not None:
            mesh = gpd.clip(mesh, subDom.to_crs(epsgOut))
        
        mesh.to_file(os.path.join(os.path.dirname(pathOut), f'{meshName}.shp'))
        print(f'    Mesh exported: {(time.time() - t0)/60:0.3f} min')
        print(f'Ready with exporting code after: {(time.time() - t00)/60:0.3f} min')
        return gdf, mesh
    
    else:
        print(f'Ready with exporting code after: {(time.time() - t00)/60:0.3f} min')
        return gdf


In [5]:
## path of the adcirc maxele output file, must be a netcdf file
ncFile ='/Users/vivianhuang/Documents/GitHub/Kalpana/examples/test/fort.63.nc' 

## name of the maxele variable to downscale. Always 'zeta_max' for downscaling
var = 'zeta'

## Contour levels. Min, Max and Step. Max IS included as in np.arange method. Values must be in vUnitOut vertical unit.
## from 0 to 3 meters (included) every 0.5
levels = [-1, 19, 0.1]

## 'polyline' or 'polygon'
## we are creating polygons in this example
conType = 'polygon'



## coordinate system of the output shapefile
epsgOut = 4326  # output in latitude and longitude, based on downscaling DEM

## input and output vertical units. For the momment only supported 'm' and 'ft'  
vUnitIn = 'm' ## Default 'm'
vUnitOut = 'm' ## Default 'ft'

## coordinate system of the adcirc input.
## Default is 4326 since ADCIRC uses latitude and longitude
epsgIn = 4326  

## complete path of the subdomain polygon kml or shapelfile, or list with the
## upper-left x, upper-left y, lower-right x and lower-right y coordinates. 
## the crs must be the same of the adcirc input file. 
subDomain = None  ## Default None

## True for export the mesh geodataframe and also save it as a shapefile. 
## for this example we are only exporting the contours, not the mesh.
exportMesh = False  ## Default False

## file name of the output mesh shapefile. Default None
meshName = None  ## Default None

## full path of the pickle file with the vertical difference between datums for each mesh node. 
dzFile = None  ## Default None

## threshold for using nearest neighbor interpolation to change datum. Points below this value won't be changed.
zeroDif = -20  ## Default -20

In [6]:
max_time_index = int(nc2['zeta'].shape[0])

In [7]:
time_index = 0
#while time_index < max_time_index:
while time_index < 10:
    output_raster_dir = '/Users/vivianhuang/Desktop/mesh-try/time_index_'+str(time_index)+'/'
    #output_raster_dir = '/Volumes/HDDLiting/neches_output/time_index_'+str(time_index)+'/'
    if not os.path.exists(output_raster_dir):
        os.makedirs(output_raster_dir)
    
    ## complete path of the output file (*.shp or *.gpkg)
    pathOut = output_raster_dir+'neches_fort63.shp'
    
    
    gdf = nc2shp63(ncFile, var, levels, conType, pathOut, epsgOut, vUnitOut=vUnitOut, vUnitIn=vUnitIn, epsgIn=epsgIn,
               subDomain=subDomain, exportMesh=exportMesh, meshName=meshName, dzFile=dzFile, zeroDif=zeroDif,time_index=time_index)
    time_index = time_index+1

Start exporting adcirc to shape


Compute contours using Dask: 100%|██████████████| 89/89 [00:01<00:00, 84.10it/s]


    Ready with the contours extraction: 0.022 min
    Saving file: 0.002 min
Ready with exporting code after: 0.025 min
Start exporting adcirc to shape


Compute contours using Dask: 100%|██████████████| 89/89 [00:01<00:00, 81.15it/s]


    Ready with the contours extraction: 0.020 min
    Saving file: 0.002 min
Ready with exporting code after: 0.023 min
Start exporting adcirc to shape


...

    Ready with the contours extraction: 0.021 min
    Saving file: 0.002 min
Ready with exporting code after: 0.023 min
Start exporting adcirc to shape


Compute contours using Dask: 100%|██████████████| 89/89 [00:01<00:00, 84.94it/s]


    Ready with the contours extraction: 0.020 min
    Saving file: 0.002 min
Ready with exporting code after: 0.022 min
Start exporting adcirc to shape


Compute contours using Dask: 100%|██████████████| 89/89 [00:01<00:00, 78.35it/s]

    Ready with the contours extraction: 0.021 min
    Saving file: 0.002 min
Ready with exporting code after: 0.023 min


In [29]:
time_index = 201
#while time_index < max_time_index:
while time_index < 202:
    output_raster_dir = '/Users/vivianhuang/Desktop/mesh-try/'
    #output_raster_dir = '/Volumes/HDDLiting/neches_output/time_index_'+str(time_index)+'/'
    if not os.path.exists(output_raster_dir):
        os.makedirs(output_raster_dir)
    
    ## complete path of the output file (*.shp or *.gpkg)
    pathOut = output_raster_dir+'neches_fort63.shp'
    
    
    gdf = nc2shp63(ncFile, var, levels, conType, pathOut, epsgOut, vUnitOut=vUnitOut, vUnitIn=vUnitIn, epsgIn=epsgIn,
               subDomain=subDomain, exportMesh=exportMesh, meshName=meshName, dzFile=dzFile, zeroDif=zeroDif,time_index=time_index)
    time_index = time_index+1

Start exporting adcirc to shape


Compute contours using Dask: 100%|██████████████| 89/89 [01:10<00:00,  1.27it/s]


    Ready with the contours extraction: 1.176 min
    Saving file: 0.030 min
    Exporting mesh
    Mesh exported: 0.122 min
Ready with exporting code after: 1.329 min


In [9]:
print(max_time_index)

384


In [10]:
for i in range(10,20):
    print(i)

10
11
12
13
14
15
16
17
18
19
